In [3]:
import numpy as np

from numpy.testing import assert_allclose

np.set_printoptions(precision=3)

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [4]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
        
    N = vec.shape[0] 
    
    y = np.zeros((N,1), dtype=float)  #Вектор у, который должен получиться 
    y[0] = np.linalg.norm(vec) 
    
    u = (vec - y.T) / np.linalg.norm(vec - y.T) #Вектор нормали и
    
    H = np.eye(N) - 2*u*u.T #Householder transformation
    
    return np.dot(vec, H), H

In [5]:
v = np.array([1, 2, 3])
householder(v)

(array([ 3.742e+00,  0.000e+00, -1.110e-16]), array([[ 0.267,  0.535,  0.802],
        [ 0.535,  0.61 , -0.585],
        [ 0.802, -0.585,  0.123]]))

Test your function using tests below:

In [6]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

In [7]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [8]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    Q = np.eye(m)
    
    # Рассматриваем случаи для разных прямоугольных матриц
    if n > m:
        k = m-1
    else:
        k = n
     
    #Срезаем столбцы матрицы для поворота
    for i in range(k):
        c = np.zeros(m-i)
        for j in range(m-i):
            c[j] = a1[i+j][i]
           
        #Повернутый вектор и матрица поворота размерности m-i
        c, h = householder(c)
        
        #Возвращаем H нужную размерность
        H = np.eye(m)
        for j in range(m-i):
            for s in range(m-i):
                H[j+i,s+i] = h[j,s]
        
        #Применяем H к а1
        a1 = H@a1
        
        #Собираем ортогональную матрицу
        Q = Q@H
        
    return Q, a1

In [9]:
a = np.random.random(size=(3, 5))
q, r = qr_decomp(a)
assert_allclose(np.dot(q, r), a)

In [10]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [11]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [12]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here* (10% of the total grade, peer-graded)

In [13]:
print(q, "\n")
print(qq, "\n")

[[ 0.137  0.536 -0.094  0.77   0.305]
 [ 0.56   0.094 -0.533  0.018 -0.627]
 [ 0.197  0.659  0.601 -0.324 -0.246]
 [ 0.625 -0.504  0.521  0.285  0.048]
 [ 0.488  0.122 -0.272 -0.47   0.672]] 

[[-0.137  0.536  0.094  0.662 -0.497]
 [-0.56   0.094  0.533 -0.525 -0.343]
 [-0.197  0.659 -0.601 -0.379  0.148]
 [-0.625 -0.504 -0.521  0.19  -0.218]
 [-0.488  0.122  0.272  0.328  0.752]] 



In [14]:
print(r, "\n")
print(rr, "\n")

[[ 1.402  1.251  0.895]
 [ 0.     0.842  0.684]
 [ 0.    -0.     0.55 ]
 [ 0.     0.     0.   ]
 [ 0.    -0.    -0.   ]] 

[[-1.402 -1.251 -0.895]
 [ 0.     0.842  0.684]
 [ 0.     0.    -0.55 ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]] 



In [15]:
assert_allclose(qq@rr, q@r)

Матрицы `r` и `rr` отличаются только знаками (по строкам).

Посмотрим на левые квадратные части матриц `q` и `qq` (блоки $n\times n$): они тоже отличаются только знаками (по столбцам). 

Заметим, что при перемножении `q` на `r` и `qq` на `rr` эта разница в знаках не меняет получаемую матрицу. Оставшиеся же правые блоки `q` и `qq` матриц совсем разные, но они умножаются на нижние нулевые блоки матриц `r` и `rr`, так что это различие значения не имеет.

Таким образом, $QR$ разложение неединтсвенно.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [16]:
def qr_dec(a): #Выдает матрицу R и список векторов u
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    reflections = []
    
        # Рассматриваем случаи для разных прямоугольных матриц
    if n > m:
        k = m-1
    else:
        k = n
    #print(a1) 
    #Срезаем столбцы матрицы для поворота
    for i in range(k):
        c = np.zeros(m-i)
        for j in range(m-i):
            c[j] = a1[i+j][i]
         
        N = c.shape[0] 
    
        y = np.zeros((N,1), dtype=float)  #Вектор у, который должен получиться 
        y[0] = np.linalg.norm(c) 
    
        u = (c - y.T) / np.linalg.norm(c - y.T) #Вектор нормали и
        
        #Вернем ему размерность m
        u1 = np.zeros(m)
        for j in range(m-i):
            u1[i+j] = u[0][j]
        
        reflections.append(u1) #Сохранили вектор нормали
        
        #Матрица R
        a1 = a1 - 2*np.outer(u1,np.dot(u1,a1)) #Вот здесь сначала умножается u^T на A, потом к ним домножается u
        
    return a1, reflections

In [17]:
qr_dec(a)

(array([[ 1.402,  1.251,  0.895],
        [ 0.   ,  0.842,  0.684],
        [-0.   ,  0.   ,  0.55 ],
        [ 0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   , -0.   ]]),
 [array([-0.657,  0.426,  0.15 ,  0.476,  0.371]),
  array([ 0.   , -0.528,  0.74 , -0.11 ,  0.402]),
  array([ 0.   ,  0.   , -0.791,  0.365, -0.491])])

In [22]:
def Q_T(a, reflections): #Домножает матрицу на Q.T, используя векторы нормали
    a1 = np.array(a, copy=True, dtype=float)
    for i in range(len(reflections)):
        u = reflections[i]
        a1 = a1 - 2*np.outer(u,np.dot(u,a1)) #Та же логика с порядком умножения, что и в предыдущем методе
    return a1

In [23]:
R, reflections = qr_dec(a)
Q_T(a, reflections)

array([[ 1.402,  1.251,  0.895],
       [ 0.   ,  0.842,  0.684],
       [-0.   ,  0.   ,  0.55 ],
       [ 0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.   ]])

Тесты:

In [24]:
# Test III.1
a = np.random.random(size=(3, 5))
R, reflections = qr_dec(a)
R1 = Q_T(a, reflections)

assert_allclose(R, R1)

In [25]:
# Test III.2
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
R, reflections = qr_dec(a)
R1 = Q_T(a, reflections)

assert_allclose(R, R1)